### Домашнее задание №4.

***Загрузим необходимые библиотеки:***

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklift.metrics import uplift_at_k
from sklift.models import SoloModel, TwoModels, ClassTransformation
from catboost import CatBoostClassifier

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

***Загрузим данные:***

In [3]:
df = pd.read_csv('./data.csv')
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


In [4]:
df.shape

(64000, 9)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


In [6]:
df.describe()

,recency,history,used_discount,used_bogo,is_referral,conversion
count,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000
mean,5.763734,242.085656,0.551031,0.549719,0.502250,0.146781
std,3.507592,256.158608,0.497393,0.497526,0.499999,0.353890
min,1.000000,29.990000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,64.660000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,158.110000,1.000000,1.000000,1.000000,0.000000
75%,9.000000,325.657500,1.000000,1.000000,1.000000,0.000000
max,12.000000,3345.930000,1.000000,1.000000,1.000000,1.000000


***Переименуем поля: (conversion -> target, offer -> treatment).***

In [7]:
df = df.rename(columns={'conversion': 'target', 'offer': 'treatment'})
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0


***Приведём поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает 
отсутствие коммуникации, а все остальные - наличие.***

In [8]:
df['treatment'] = np.where(df['treatment'] != 'No Offer', 1, 0)
df.head(3)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0


In [9]:
df['treatment'].unique()

array([1, 0])

***Посмотрим соотношение  значений признаков 'treatment' и 'target':***

In [10]:
df[['treatment', 'target']].value_counts()

treatment  target
1          0         35562
0          0         19044
1          1          7132
0          1          2262
dtype: int64

***Переименуем индексы в id:***

In [11]:
df['id'] = df.index
df.set_index('id', inplace=True)
df.head(3)


,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
id,,,,,,,,,
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0


In [12]:
df['channel'].unique()

array(['Phone', 'Web', 'Multichannel'], dtype=object)

In [13]:
df['zip_code'].unique()

array(['Surburban', 'Rural', 'Urban'], dtype=object)

***Сгенерируем новые признаки:*** 

In [14]:
cols_to_drop = ['id', 'zip_code', 'channel', 'treatment', 'target']
cols = list(set(df.columns) - set(cols_to_drop))
base_cols = cols

In [15]:
base_cols

['is_referral', 'used_discount', 'recency', 'used_bogo', 'history']

In [16]:
def transform_data(data):
    for i1, col1 in enumerate(base_cols):
        data[col1 + '_log'] = np.log(data[col1] + 1.1)

        for i2, col2 in enumerate(base_cols):
            data[f'{col1}_{col2}_1'] = data[col1] - data[col2]
            data[f'{col1}_{col2}_2'] = data[col1] + data[col2]
            data[f'{col1}_{col2}_3'] = data[col1] / (data[col2] + 0.1)
            data[f'{col1}_{col2}_4'] = data[col1] * data[col2]

            data[f'{col1}_{col2}_11'] = data[col1] - np.log(data[col2] + 1)
            data[f'{col1}_{col2}_22'] = data[col1] + np.log(data[col2] + 1)
            data[f'{col1}_{col2}_33'] = data[col1] / (np.log(data[col2] + 1) + 0.1)
            data[f'{col1}_{col2}_44'] = data[col1] * np.log(data[col2] + 1)

    return data


df_t = transform_data(df)

In [17]:
df_t.head(10)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target,is_referral_log,is_referral_is_referral_1,is_referral_is_referral_2,is_referral_is_referral_3,is_referral_is_referral_4,is_referral_is_referral_11,is_referral_is_referral_22,is_referral_is_referral_33,is_referral_is_referral_44,is_referral_used_discount_1,is_referral_used_discount_2,is_referral_used_discount_3,is_referral_used_discount_4,is_referral_used_discount_11,is_referral_used_discount_22,is_referral_used_discount_33,is_referral_used_discount_44,is_referral_recency_1,is_referral_recency_2,is_referral_recency_3,is_referral_recency_4,is_referral_recency_11,is_referral_recency_22,is_referral_recency_33,is_referral_recency_44,is_referral_used_bogo_1,is_referral_used_bogo_2,is_referral_used_bogo_3,is_referral_used_bogo_4,is_referral_used_bogo_11,is_referral_used_bogo_22,is_referral_used_bogo_33,is_referral_used_bogo_44,is_referral_history_1,is_referral_history_2,is_referral_history_3,is_referral_history_4,is_referral_history_11,is_referral_history_22,is_referral_history_33,is_referral_history_44,...,used_bogo_used_bogo_44,used_bogo_history_1,used_bogo_history_2,used_bogo_history_3,used_bogo_history_4,used_bogo_history_11,used_bogo_history_22,used_bogo_history_33,used_bogo_history_44,history_log,history_is_referral_1,history_is_referral_2,history_is_referral_3,history_is_referral_4,history_is_referral_11,history_is_referral_22,history_is_referral_33,history_is_referral_44,history_used_discount_1,history_used_discount_2,history_used_discount_3,history_used_discount_4,history_used_discount_11,history_used_discount_22,history_used_discount_33,history_used_discount_44,history_recency_1,history_recency_2,history_recency_3,history_recency_4,history_recency_11,history_recency_22,history_recency_33,history_recency_44,history_used_bogo_1,history_used_bogo_2,history_used_bogo_3,history_used_bogo_4,history_used_bogo_11,history_used_bogo_22,history_used_bogo_33,history_used_bogo_44,history_history_1,history_history_2,history_history_3,history_history_4,history_history_11,history_history_22,history_history_33,history_history_44
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,10,142.44,1,0,Surburban,0,Phone,1,0,0.095310,0,0,0.000000,0,0.000000,0.000000,0.0000,0.000000,-1,1,0.000000,0,-0.693147,0.693147,0.0000,0.000000,-10,10,0.000000,0,-2.397895,2.397895,0.000000,0.000000,0,0,0.000000,0,0.000000,0.000000,0.0000,0.000000,-142.44,142.44,0.000000,0.00,-4.965917,4.965917,0.000000,0.000000,...,0.000000,-142.44,142.44,0.000000,0.00,-4.965917,4.965917,0.000000,0.000000,4.966614,142.44,142.44,1424.400000,0.00,142.440000,142.440000,1424.400000,0.000000,141.44,143.44,129.490909,142.44,141.746853,143.133147,179.588358,98.731884,132.44,152.44,14.102970,1424.40,140.042105,144.837895,57.024008,341.556203,142.44,142.44,1424.400000,0.00,142.440000,142.440000,1424.400000,0.000000,0.0,284.88,0.999298,20289.1536,137.474083,147.405917,28.117319,707.345193
1,6,329.08,1,1,Rural,1,Web,0,0,0.741937,0,2,0.909091,1,0.306853,1.693147,1.2608,0.693147,0,2,0.909091,1,0.306853,1.693147,1.2608,0.693147,-5,7,0.163934,6,-0.945910,2.945910,0.488780,1.945910,0,2,0.909091,1,0.306853,1.693147,1.2608,0.693147,-328.08,330.08,0.003038,329.08,-4.799335,6.799335,0.169511,5.799335,...,0.693147,-328.08,330.08,0.003038,329.08,-4.799335,6.799335,0.169511,5.799335,5.799638,328.08,330.08,299.163636,329.08,328.386853,329.773147,414.904078,228.100874,328.08,330.08,299.163636,329.08,328.386853,329.773147,414.904078,228.100874,323.08,335.08,53.947541,1974.48,327.134090,331.025910,160.847728,640.360112,328.08,330.08,299.163636,329.08,328.386853,329.773147,414.904078,228.100874,0.0,658.16,0.999696,108293.6464,323.280665,334.879335,55.782558,1908.445178
2,7,180.65,0,1,Surburban,1,Web,1,0,0.741937,0,2,0.909091,1,0.306853,1.693147,1.2608,0.693147,1,1,10.000000,0,1.000000,1.000000,10.0000,0.000000,-6,8,0.140845,7,-1.079442,3.079442,0.458833,2.079442,0,2,0.

In [18]:
df_t.shape

(64000, 214)

***Разделим датасет на трейн и тест:***

In [19]:
train, test = train_test_split(df_t, test_size=0.3, random_state=42)
#Делаем данные для обучения
y_train = train.loc[train.index, 'target']
treat_train = train.loc[train.index, 'treatment']
train_features = train.drop(['target', 'treatment'], axis=1)
#То же самое нам нужно для тестовых данных
y_test = test.loc[test.index, 'target']
treat_test = test.loc[test.index, 'treatment']
test_features = test.drop(['target', 'treatment'], axis=1)

***Обозначим категориальные признаки для CatBoosta и подготовим словарь для результатов работы моделей:***

In [20]:
cat_features = ['zip_code', 'channel']

models_results = {
    'model': [],
    'uplift@10%': [],
    'uplift@20%': []
}

***Обучим наши модели:***

In [21]:
#Соло модель
sm = SoloModel(
    CatBoostClassifier(n_estimators=300, max_depth=5, 
                       auto_class_weights='Balanced', 
                       silent=True, random_state=42, learning_rate=0.1,
                       early_stopping_rounds=20, cat_features=cat_features)
)

sm = sm.fit(train_features, y_train, treat_train)

uplift_sm = sm.predict(test_features)

sm10_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)
sm20_score = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.2)
models_results['model'].append('SoloModel')
models_results['uplift@10%'].append(sm10_score)
models_results['uplift@20%'].append(sm20_score)

In [22]:
#Трансформация классов
ct = ClassTransformation(
    CatBoostClassifier(n_estimators=300, max_depth=5, 
                       auto_class_weights='Balanced', 
                       silent=True, random_state=42, learning_rate=0.1,
                       early_stopping_rounds=20, cat_features=cat_features)
)
ct = ct.fit(train_features, y_train, treat_train)

uplift_ct = ct.predict(test_features)

ct10_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.1)
ct20_score = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.2)

models_results['model'].append('ClassTransformation')
models_results['uplift@10%'].append(ct10_score)
models_results['uplift@20%'].append(ct20_score)

In [23]:
#Две независимые модели
tm = TwoModels(
    CatBoostClassifier(n_estimators=300, max_depth=5, 
                       auto_class_weights='Balanced', 
                       silent=True, random_state=42, learning_rate=0.1,
                       early_stopping_rounds=20, cat_features=cat_features),
    CatBoostClassifier(n_estimators=300, max_depth=5, 
                       auto_class_weights='Balanced', 
                       silent=True, random_state=42, learning_rate=0.1,
                       early_stopping_rounds=20, cat_features=cat_features),
    method='vanilla'  # независимые модели
)
tm = tm.fit(train_features, y_train, treat_train)

uplift_tm = tm.predict(test_features)

tm10_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
tm20_score = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
models_results['model'].append('TwoModels')
models_results['uplift@10%'].append(tm10_score)
models_results['uplift@20%'].append(tm20_score)

***Выведем результаты в виде таблицы:***

In [24]:
pd.DataFrame(data=models_results).sort_values('uplift@10%', ascending=False)

,model,uplift@10%,uplift@20%
1,ClassTransformation,0.112784,0.087790
2,TwoModels,0.085834,0.085834
0,SoloModel,0.065346,0.068592


***Модель ClassTransformation оказалась лучше всех. Не ожидал... При оптимизации параметров CatBoosta Разница только увеличивается. Так и не понял почему.***